# Pares con mal TMT

In [4]:
import json

# Ruta de tu archivo original
input_path  = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json'
# Ruta donde quieres guardar el filtrado
output_path = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json'

# 1. Cargar JSON original
with open(input_path, 'r') as f:
    data = json.load(f)

# 2. Lista de escenarios considerados “Buenos”
malos = [
    "scenario_2",
    "scenario_3",
    "scenario_5",
    "scenario_6",
    "scenario_7",
    "scenario_8",
    "scenario_9",
    "scenario_10",
    "scenario_11",
    "scenario_12",
    "scenario_13",
    "scenario_14",
    "scenario_15",
    "scenario_16",
    "scenario_17",
    "scenario_18",
    "scenario_19",
    "scenario_20",
    "scenario_37",
    "scenario_38",
    "scenario_39",
    "scenario_40",
    "scenario_41",
    "scenario_42",
    "scenario_43",
    "scenario_44",
    "scenario_45",
    "scenario_46",
    "scenario_47",
    "scenario_48",
    "scenario_49",
    "scenario_50",
    "scenario_51",
    "scenario_52"
]

# 3. Filtrar sólo las entradas cuyo 'scenario_id' esté en la lista de buenos
filtered = [item for item in data if item.get('scenario_id') in malos]

# 4. Guardar el resultado en el nuevo JSON
with open(output_path, 'w') as f:
    json.dump(filtered, f, indent=2)

print(f'Se han guardado {len(filtered)} pares en:\n{output_path}')


Se han guardado 3400 pares en:
/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json


# Bad TMT

In [6]:
import json
import os
import math
import copy # Para crear copias de los diccionarios si es necesario

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)
TARGET_SCENARIO_ID = "scenario_2" # Escenario específico a analizar

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json"

print(f"Archivo de entrada: {input_file}")
print(f"Analizando SOLAMENTE para: '{TARGET_SCENARIO_ID}'")

# Listas para clasificar los pares del escenario objetivo
coordinated_pairs = []
uncoordinated_pairs = []

try:
    # 1. Cargar el archivo JSON (lista de pares)
    print("Cargando datos (lista de pares)...")
    with open(input_file, 'r') as f:
        relay_pairs_data = json.load(f)
    print("Datos cargados correctamente.")

    # Validar que es una lista
    if not isinstance(relay_pairs_data, list):
        raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")

    # 2. Procesar cada par de relés, filtrando por scenario_id
    print(f"Calculando delta_t, mt y clasificando pares para '{TARGET_SCENARIO_ID}'...")
    total_pairs_read = 0
    scenario_pairs_found = 0
    skipped_pairs_count = 0 # Pares omitidos DENTRO del scenario_id objetivo

    for pair_entry in relay_pairs_data:
        total_pairs_read += 1
        if not isinstance(pair_entry, dict):
            # Contar como leído pero no procesado si no es diccionario
            continue

        # --- FILTRAR POR SCENARIO_ID ---
        current_scenario_id = pair_entry.get("scenario_id")
        if current_scenario_id != TARGET_SCENARIO_ID:
            continue # Omitir este par si no es del escenario objetivo
        # ---------------------------------

        # Si llegamos aquí, el par pertenece a TARGET_SCENARIO_ID
        scenario_pairs_found += 1

        # Obtener información de los relés de forma segura
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Falta información de relé principal o de respaldo en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')}. Par omitido.")
            skipped_pairs_count += 1
            continue

        # Obtener tiempos de operación de forma segura
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Validar que los tiempos son números
        if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Tiempo(s) de operación no numéricos o faltantes en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')} (Main: {main_time}, Backup: {backup_time}). Par omitido.")
            skipped_pairs_count += 1
            continue

        # --- Realizar Cálculos ---
        delta_t = backup_time - main_time - CTI
        mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo

        # Crear una copia del par y añadirle los resultados del cálculo
        pair_info = copy.deepcopy(pair_entry)
        pair_info['delta_t'] = delta_t
        pair_info['mt'] = mt

        # --- Clasificar (solo pares del TARGET_SCENARIO_ID) ---
        if delta_t >= 0:
            coordinated_pairs.append(pair_info)
        else:
            uncoordinated_pairs.append(pair_info)

    print("Procesamiento de pares completado.")

    # 3. Calcular Métricas Finales (solo para TARGET_SCENARIO_ID)
    if scenario_pairs_found == 0:
         print (f"No se encontraron pares válidos para '{TARGET_SCENARIO_ID}' en el archivo.")
    else:
        total_valid_pairs_scenario = len(coordinated_pairs) + len(uncoordinated_pairs) # Pares válidos DENTRO del escenario
        miscoordination_count_scenario = len(uncoordinated_pairs)
        # Sumar 'mt' solo de los pares del escenario objetivo (que son los únicos en las listas)
        tmt_total_scenario = sum(pair["mt"] for pair in coordinated_pairs + uncoordinated_pairs)


        # 4. Imprimir Resultados
        print(f"\n--- Resultados del Análisis de Coordinación para '{TARGET_SCENARIO_ID}' ---")
        print(f"Total de pares leídos del archivo: {total_pairs_read}")
        print(f"Total de pares encontrados para '{TARGET_SCENARIO_ID}': {scenario_pairs_found}")
        if skipped_pairs_count > 0:
            print(f"Pares omitidos DENTRO de '{TARGET_SCENARIO_ID}' (datos inválidos/faltantes): {skipped_pairs_count}")
        print(f"Total de pares válidos analizados para '{TARGET_SCENARIO_ID}': {total_valid_pairs_scenario}")
        print(f"Número de pares coordinados (delta_t >= 0) en '{TARGET_SCENARIO_ID}': {len(coordinated_pairs)}")
        print(f"Número de pares DESCOORDINADOS (delta_t < 0) en '{TARGET_SCENARIO_ID}': {miscoordination_count_scenario}")
        print(f"Suma total de penalización por descoordinación (TMT Total) para '{TARGET_SCENARIO_ID}': {tmt_total_scenario:.5f}") # Imprimir con 5 decimales

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
except TypeError as e:
    print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json
Analizando SOLAMENTE para: 'scenario_2'
Cargando datos (lista de pares)...
Datos cargados correctamente.
Calculando delta_t, mt y clasificando pares para 'scenario_2'...
Procesamiento de pares completado.

--- Resultados del Análisis de Coordinación para 'scenario_2' ---
Total de pares leídos del archivo: 3400
Total de pares encontrados para 'scenario_2': 100
Total de pares válidos analizados para 'scenario_2': 100
Número de pares coordinados (delta_t >= 0) en 'scenario_2': 90
Número de pares DESCOORDINADOS (delta_t < 0) en 'scenario_2': 10
Suma total de penalización por descoordinación (TMT Total) para 'scenario_2': -13.95800


# Analisys

In [7]:

import json
import os
import math
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import logging
import numpy as np # Para isnan
from typing import Dict, List, Any, Optional

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)

# --- Rutas de los archivos ---
# ENTRADA: Archivo con la lista de pares, incluyendo 'Time_out' en cada relé
INPUT_FILE = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json"
# SALIDA (Opcional): Gráfico HTML
# PLOT_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/tmt_analysis_all_scenarios_from_input_times.html"

logger.info(f"Archivo de entrada: {INPUT_FILE}")

# %%
# Celda 2: Funciones de Ayuda (Carga)

def load_json_file(file_path: str) -> Optional[Any]:
    """Loads data from a JSON file."""
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

# %%
# Celda 3: Carga y Agrupación de Datos por Escenario

logger.info("Cargando y agrupando datos por escenario...")
relay_pairs_data = load_json_file(INPUT_FILE)

if relay_pairs_data is None:
    raise SystemExit("Error crítico al cargar archivo de entrada.")
if not isinstance(relay_pairs_data, list):
     raise SystemExit("Error de formato en archivo de entrada (se espera lista).")

# Crear mapa de pares originales: scenario_id -> [list of pairs]
original_pairs_map: Dict[str, List[Dict]] = {}
total_original_pairs_read = 0
skipped_pairs_structure = 0
for pair in relay_pairs_data:
    s_id = pair.get("scenario_id")
    # Validar estructura mínima y la presencia de Time_out antes de agrupar
    main_info = pair.get('main_relay')
    backup_info = pair.get('backup_relay')
    if s_id and isinstance(main_info, dict) and isinstance(backup_info, dict) and \
       'Time_out' in main_info and 'Time_out' in backup_info:
        if s_id not in original_pairs_map: original_pairs_map[s_id] = []
        original_pairs_map[s_id].append(pair)
        total_original_pairs_read += 1
    else:
        skipped_pairs_structure += 1
        # logger.warning(f"Par omitido por estructura/Time_out inválido: {pair.get('scenario_id', 'N/A')}")

logger.info(f"Datos agrupados: {len(original_pairs_map)} escenarios encontrados.")
logger.info(f"{total_original_pairs_read} pares válidos leídos y agrupados.")
if skipped_pairs_structure > 0:
    logger.warning(f"{skipped_pairs_structure} pares fueron omitidos durante la carga debido a estructura o Time_out faltante.")


# %%
# Celda 4: Calcular Métricas de Coordinación para Todos los Escenarios (Usando Tiempos Existentes)

logger.info("Calculando TMT y métricas para cada escenario usando los Time_out existentes...")
scenario_results: Dict[str, Dict[str, Any]] = {}

if not original_pairs_map:
     raise SystemExit("No hay datos de escenario para procesar.")

for scenario_id, pairs_for_scenario in original_pairs_map.items():
    tmt_for_scenario = 0.0
    miscoordination_count = 0
    valid_pairs_evaluated = 0
    max_neg_mt_scenario = 0.0
    skipped_this_scenario = 0

    for pair_entry in pairs_for_scenario:
        main_relay_info = pair_entry.get('main_relay') # Ya validado que existe
        backup_relay_info = pair_entry.get('backup_relay') # Ya validado que existe

        # Obtener tiempos existentes
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Validar que los tiempos son números válidos
        if not isinstance(main_time, (int, float)) or math.isnan(main_time) or \
           not isinstance(backup_time, (int, float)) or math.isnan(backup_time):
            # logger.warning(f"({scenario_id}) Tiempos inválidos en par {main_relay_info.get('relay')}/{backup_relay_info.get('relay')}. Omitido.")
            skipped_this_scenario += 1
            continue

        # --- Calcular delta_t y mt con tiempos existentes ---
        delta_t = backup_time - main_time - CTI
        mt = min(0, delta_t) # Penalización solo si delta_t es negativo

        valid_pairs_evaluated += 1
        if mt < -1e-6: # Usar tolerancia
            miscoordination_count += 1
            tmt_for_scenario += mt
            max_neg_mt_scenario = min(max_neg_mt_scenario, mt)

    # Almacenar resultados del escenario
    if valid_pairs_evaluated > 0:
        scenario_results[scenario_id] = {
            "TMT": tmt_for_scenario,
            "MiscoordinatedCount": miscoordination_count,
            "TotalPairsEvaluated": valid_pairs_evaluated,
            "MaxNegativeMT": max_neg_mt_scenario
        }
        # Log resumido por escenario
        # logger.info(f"Resultados {scenario_id}: TMT={tmt_for_scenario:.4f}, Descoordinados={miscoordination_count}/{valid_pairs_evaluated}")
        if skipped_this_scenario > 0:
             logger.warning(f"({scenario_id}) Se omitieron {skipped_this_scenario} pares adicionales durante el cálculo por tiempos inválidos.")
    elif pairs_for_scenario: # Si había pares pero ninguno fue válido
         logger.warning(f"({scenario_id}) No se evaluaron pares válidos (problemas con Time_out?).")


logger.info(f"Cálculo de TMT completado para {len(scenario_results)} escenarios.")

# %%
# Celda 5: Análisis - Mostrar los 54 Mejores Escenarios por TMT

if not scenario_results:
    logger.error("No hay resultados de escenario para analizar.")
else:
    # Convertir a DataFrame
    df_results = pd.DataFrame.from_dict(scenario_results, orient='index')
    df_results.index.name = 'scenario_id'
    df_results = df_results.reset_index()

    # Ordenar por TMT (descendente: mejor primero)
    df_results_sorted_by_tmt = df_results.sort_values(by='TMT', ascending=False)

    # Seleccionar los 54 mejores diferentes de cero
    top_54_scenarios = df_results_sorted_by_tmt[df_results_sorted_by_tmt['TMT'] != 0].head(54)
    # top_54_scenarios = df_results_sorted_by_tmt.head(54)

    print("\n--- Mejores 54 Escenarios por TMT (Basado en Tiempos del Archivo de Entrada) ---")
    pd.set_option('display.max_rows', 60) # Mostrar suficientes filas
    # Mostrar columnas relevantes
    print(top_54_scenarios[['scenario_id', 'TMT', 'MiscoordinatedCount', 'TotalPairsEvaluated', 'MaxNegativeMT']].round(5))
    pd.reset_option('display.max_rows')

    # Guardar en CSV (Opcional)
    # try:
    #     top_54_scenarios.to_csv("top_54_scenarios_by_tmt_input_times.csv", index=False)
    #     logger.info("Datos de los 54 mejores escenarios guardados en top_54_scenarios_by_tmt_input_times.csv")
    # except Exception as e:
    #     logger.error(f"No se pudo guardar el CSV: {e}")


# %%
# Celda 6: Visualización - Gráfico de Barras del TMT para Todos los Escenarios

if not scenario_results:
    logger.error("No hay resultados de TMT para graficar.")
elif 'df_results' not in locals():
     logger.error("DataFrame df_results no fue creado. No se puede graficar.")
else:
    logger.info("Generando gráfico de barras de TMT para todos los escenarios...")

    # Ordenar por ID de escenario para el gráfico
    def get_scenario_num(s_id):
        try: return int(s_id.split('_')[-1])
        except: return float('inf') # Poner nombres no estándar al final

    df_results_plot_sorted = df_results.copy()
    df_results_plot_sorted['scenario_num'] = df_results_plot_sorted['scenario_id'].apply(get_scenario_num)
    df_results_plot_sorted = df_results_plot_sorted.sort_values(by='scenario_num')

    # Extraer datos ordenados
    scenario_labels_plot = df_results_plot_sorted['scenario_id'].tolist()
    tmt_values_plot = df_results_plot_sorted['TMT'].tolist()
    miscoord_counts_plot = df_results_plot_sorted['MiscoordinatedCount'].tolist()
    total_pairs_plot = df_results_plot_sorted['TotalPairsEvaluated'].tolist()
    max_neg_mt_plot = df_results_plot_sorted['MaxNegativeMT'].tolist()

    # Textos de hover
    hover_texts_plot = [
        f"<b>{s_id}</b><br>" +
        f"TMT: {tmt:.4f}<br>" +
        f"Descoordinados: {mc}/{tp}<br>" +
        f"Peor MT Individual: {max_neg_mt:.4f}"
        for s_id, tmt, mc, tp, max_neg_mt in zip(scenario_labels_plot, tmt_values_plot, miscoord_counts_plot, total_pairs_plot, max_neg_mt_plot)
    ]

    # Colores (ajustar umbrales si es necesario)
    colors = ['#2ca02c' if tmt >= -0.001 else '#ff7f0e' if tmt >= -0.05 else '#d62728' # Verde, Naranja, Rojo
              for tmt in tmt_values_plot]

    # Crear figura
    fig_tmt_all = go.Figure(data=[
        go.Bar(
            x=scenario_labels_plot,
            y=tmt_values_plot,
            text=[f"{mc}/{tp}" if tp > 0 else "N/A" for mc, tp in zip(miscoord_counts_plot, total_pairs_plot)], # Mostrar Descoord/Total
            textposition='outside',
            textfont_size=8,
            hovertext=hover_texts_plot,
            hoverinfo='text',
            marker_color=colors
        )
    ])

    fig_tmt_all.update_layout(
        title="TMT Total por Escenario (Basado en Tiempos del Archivo de Entrada)",
        xaxis_title="ID de Escenario",
        yaxis_title="TMT Total (Suma de mt < 0) [s]",
        yaxis=dict(range=[min(tmt_values_plot + [-0.1]) * 1.1, max(tmt_values_plot + [0.05])]), # Ajustar rango eje Y
        yaxis_zeroline=True, yaxis_zerolinewidth=2, yaxis_zerolinecolor='black',
        xaxis={'type': 'category', 'tickangle': -90}, # Rotar etiquetas
        height=700, # Ajustar altura si hay muchos escenarios
        bargap=0.2,
        hovermode='x unified'
    )

    # Mostrar y guardar
    fig_tmt_all.show()
    # try:
    #     output_dir = os.path.dirname(PLOT_OUTPUT_PATH)
    #     if output_dir: os.makedirs(output_dir, exist_ok=True)
    #     fig_tmt_all.write_html(PLOT_OUTPUT_PATH)
    #     logger.info(f"Gráfico guardado en: {PLOT_OUTPUT_PATH}")
    # except Exception as e:
    #     logger.error(f"No se pudo guardar el gráfico en {PLOT_OUTPUT_PATH}: {e}")


logger.info("--- Análisis de TMT para todos los escenarios (basado en tiempos de entrada) Finalizado ---")

2025-04-19 19:21:50,353 - INFO - Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json
2025-04-19 19:21:50,355 - INFO - Cargando y agrupando datos por escenario...
2025-04-19 19:21:50,812 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json
2025-04-19 19:21:50,817 - INFO - Datos agrupados: 34 escenarios encontrados.
2025-04-19 19:21:50,817 - INFO - 3400 pares válidos leídos y agrupados.
2025-04-19 19:21:50,817 - INFO - Calculando TMT y métricas para cada escenario usando los Time_out existentes...
2025-04-19 19:21:50,823 - INFO - Cálculo de TMT completado para 34 escenarios.
2025-04-19 19:21:50,849 - INFO - Generando gráfico de barras de TMT para todos los escenarios...



--- Mejores 54 Escenarios por TMT (Basado en Tiempos del Archivo de Entrada) ---
    scenario_id      TMT  MiscoordinatedCount  TotalPairsEvaluated  \
6    scenario_9 -13.3405                    9                  100   
12  scenario_15 -13.5756                   17                  100   
13  scenario_16 -13.8589                   11                  100   
0    scenario_2 -13.9580                   10                  100   
19  scenario_38 -14.0657                    9                  100   
18  scenario_37 -14.1898                    9                  100   
1    scenario_3 -14.2484                    9                  100   
17  scenario_20 -14.3601                    9                  100   
3    scenario_6 -14.4177                    9                  100   
2    scenario_5 -14.4276                   10                  100   
5    scenario_8 -14.4887                   11                  100   
28  scenario_47 -14.6540                   23                  100   
26  scen

2025-04-19 19:21:50,858 - INFO - --- Análisis de TMT para todos los escenarios (basado en tiempos de entrada) Finalizado ---
